In [4]:
import pandas as pd

### combine my 3 CSV's into one CSV

In [178]:
df = pd.concat(map(pd.read_csv, ['Tournaments_data_3.csv', 'Tournaments_data_4.csv','Tournaments_data_5.csv']))

In [179]:
df.to_csv("combined_data.csv", encoding='utf-8')

In [180]:
df = df.set_index("Unique ID")

### Raw data to be brought in after combining CSV's

In [135]:
df = pd.read_csv("combined_data.csv", index_col="Unique ID")

### Remove any null values
The null values we have are in the date column, as well as the game type column. Any date that was on the day that my script ran came back NaN, it doesn't seem to matter much to remove these as that day has past, and it will be just a few hundred lines. I am not sure why a few hundred came back without a game type, but they mostly seemed to be from the same locations. While it does suck to lose those locations, the game type is very key information, and I will be ommiting them unless I run the script again and can solve the issue of them coming back blank. This still leaves me with over 7k rows of information.

In [181]:
df = df.dropna()

### Convert date and time into readable DateTime

In [182]:
df.Date = df.Date.str[1:5]
df.Date = df.Date + "/2023"

In [183]:
df['Start time'] = df['Start time'].str.replace("am", ":00 am")
df['Start time'] = df['Start time'].str.replace("pm", ":00 pm")

In [184]:
df["Date Time"] = df["Date"] + " " + df["Start time"]

In [185]:
df = df.drop(['Date', 'Start time'], axis=1)

### Create new column for the ratio of the buy in the goes to the prize pool
This step includes some slicing, as well as replacing "day 2" with "1". It will be important to note in the future that any "Total buy in" with a value of "1" is a continuation of a tournament that started on different days.

This step also includes turning the objects into numerical values

In [187]:
df["Total buy in"] = df["Total buy in"].str[1:]
df["Prize pool"] = df["Prize pool"].str[1:]
df["rake"] = df["rake"].str[1:]

In [191]:
df["Total buy in"] = df["Total buy in"].replace("ay 2", "1")


In [192]:
df["Total buy in"] = df["Total buy in"].str.replace(",", "")
df["Prize pool"] = df["Prize pool"].str.replace(",", "")


In [193]:
df["Total buy in"] = pd.to_numeric(df["Total buy in"])
df["Prize pool"] = pd.to_numeric(df["Prize pool"])

In [198]:
df["Prize pool ratio"] = df["Prize pool"] / df["Total buy in"]

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7066 entries, 235 to 8172
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   location          7066 non-null   object 
 1   Venue             7066 non-null   object 
 2   Game type         7066 non-null   object 
 3   Total buy in      7066 non-null   int64  
 4   Prize pool        7066 non-null   int64  
 5   rake              7066 non-null   object 
 6   Guarantee         7066 non-null   object 
 7   Date Time         7066 non-null   object 
 8   Prize pool ratio  6975 non-null   float64
dtypes: float64(1), int64(2), object(6)
memory usage: 552.0+ KB


In [205]:
df["Prize pool ratio"].max()

50000.0

In [201]:
df.to_csv("testing_in_tableau.csv", encoding='utf-8')